# 12장 내장 자료형 상속과 다중 상속

이 장에서는 상속에 대해 설명하며 특히 다음 두 가지 특징을 집중적으로 다룬다.

* 내장 자료형 상속의 위험성
* 다중 상속과 메서드 결정 순서

## 12.1 내장 자료형의 상속은 까다롭다

C 언어로 작성된 내장 클래스의 코드는 사용자가 오버라이드한 코드를 호출하지 않으므로 주의가 필요하다.

In [1]:
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value]*2) 

In [5]:
dd = DoppelDict(one=1) # dict 클래스의 __init__() 메서드는 __setitem__() 이 오버라이드 되었다는 사실을 무시
dd

{'one': 1}

In [6]:
dd['two']=2 # 오버라이드된 __setitem__()을 호출하므로 [2,2]에 매핑
dd

{'one': 1, 'two': [2, 2]}

In [7]:
dd.update(three=3) # update() 메서드는 오버라이드된 __setitem__() 메서드를 호출하지 않는다
dd

{'one': 1, 'two': [2, 2], 'three': 3}

이 문제는 `self.get()`이 `self.__getitem__()`을 호출하는 경우처럼 객체 안에서 호출할 때뿐만 아니라, 내장 메서드가 호출하는 다른 클래스의 오버라이드된 메서드에서도 발생한다.

In [8]:
class AnswerDict(dict):
    def __getitem__(self, key):
        return 42

In [12]:
ad = AnswerDict(a='foo') # ad는 ('a','foo') 키-값 쌍으로 채운 AnswerDict 객체
ad['a'] # __getitem__() 은 42를 반환

42

In [13]:
d = {}
d.update(ad) # update() 메서드는 AnswerDict.__getitem__() 메서드를 무시
d['a'] 

'foo'

In [14]:
d

{'a': 'foo'}

`dict`, `list`, `str` 등의 내장 자료형은 사용자가 정의한 오버라이드된 메서드를 무시하므로, 내장 자료형보다는 `UserDict`, `UserList`, `UserString` 등을 사용하는 것이 좋다.

In [15]:
import collections

class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value]*2)

In [16]:
dd = DoppelDict2(one=1)
dd

{'one': [1, 1]}

In [17]:
dd['two'] = 2
dd

{'one': [1, 1], 'two': [2, 2]}

In [18]:
dd.update(three=3)
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

In [19]:
class AnswerDict2(collections.UserDict):
    def __getitem__(self, key):
        return 42

In [20]:
ad = AnswerDict2(a='foo')
ad['a']

42

In [21]:
d = {}
d.update(ad) 
d['a'] 

42

In [22]:
d

{'a': 42}

문제 없다!

## 12.2 다중 상속과 메서드 결정 순서